In [1]:
from transformers import AutoImageProcessor, AutoModelForObjectDetection, Trainer
import torch
import sys
import os
import pandas as pd
from pycocotools.coco import COCO
from tqdm import tqdm
from functools import partial
import numpy as np

import TTA
from test_eval import test_eval

sys.path.append("../1019_resume_train")
import utils
import dataset


In [7]:
model_path = '../1011_model_search/result/1015/jozhang97/deta-swin-large_3_img_size_720'
coco_dir_path = '../../../Split_data'
data_dir_path = '../../../../data/dataset'
test_info_name = 'test.json'
device = 'cuda'

checkpoint_path = utils.find_checkpoint_path(model_path)
run_name = os.path.split(model_path)[-1]
json_path = os.path.join(model_path, run_name + '.json')

# conf 파일 경로 덮어쓰기
conf = utils.read_json(json_path)
conf['coco_dir_path'] = coco_dir_path
conf['data_dir_path'] = data_dir_path


valid_info_path = os.path.join(conf['coco_dir_path'], conf['valid_info_name'])
coco_valid = COCO(valid_info_path)
valid = dataset.COCO2dataset(conf['data_dir_path'], coco_valid, range(10))

test_info_path = os.path.join(conf['data_dir_path'], test_info_name)
coco_test = COCO(test_info_path)
test = dataset.COCO2dataset(conf['data_dir_path'], coco_test)


# train_info_path = os.path.join(conf['coco_dir_path'], conf['train_info_name'])
# coco_train = COCO(train_info_path)
# train = dataset.COCO2dataset(conf['data_dir_path'], coco_train)

image_processor = AutoImageProcessor.from_pretrained(checkpoint_path)
model = AutoModelForObjectDetection.from_pretrained(checkpoint_path)
model.to(device)


loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


DetaForObjectDetection(
  (model): DetaModel(
    (backbone): DetaBackboneWithPositionalEncodings(
      (model): SwinBackbone(
        (embeddings): SwinEmbeddings(
          (patch_embeddings): SwinPatchEmbeddings(
            (projection): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
          )
          (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (encoder): SwinEncoder(
          (layers): ModuleList(
            (0): SwinStage(
              (blocks): ModuleList(
                (0-1): 2 x SwinLayer(
                  (layernorm_before): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
                  (attention): SwinAttention(
                    (self): SwinSelfAttention(
                      (query): Linear(in_features=192, out_features=192, bias=True)
                      (key): Linear(in_features=192, out_features=192, bias=True)
                      (value): Linear(in_feat

In [8]:
select = 'valid'
save_dir_path = f'result/TTA/{select}'
os.makedirs(save_dir_path, exist_ok=True)

tta_transforms_list = [
        (TTA.Compose([TTA.horizontal_flip]), 'hflip'),
        (TTA.Compose([TTA.identity]), 'identity')
    ]

if select == 'valid':
    for tta_transforms, tta_info in tta_transforms_list:
        save_path = f'{save_dir_path}/{run_name}_{tta_info}.csv'
        test_eval(model, image_processor, coco_valid, valid, tta_transforms, save_path)

if select == 'test':
    for tta_transforms, tta_info in tta_transforms_list:
        save_path = f'{save_dir_path}/{run_name}_{tta_info}.csv'
        test_eval(model, image_processor, coco_valid, valid, tta_transforms, save_path)


 91%|█████████ | 10/11 [00:03<00:00,  2.74it/s]


IndexError: list index out of range